# Generate rocks

In [1]:
rocks_str_raw = '''####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##
'''

In [2]:
from dataclasses import dataclass

@dataclass
class Rock:
    # +y
    # (0,0) +x
    parts: list[tuple[int, int]]
    width: int
    height: int

    def __init__(self):
        self.parts = []
        self.width = self.height = 0

    def add_part(self, new_part: tuple[int, int]):
        self.parts.append(new_part)
        self.width = max(new_part[0] + 1, self.width)
        self.height = max(new_part[1] + 1, self.height)

In [3]:
rocks_str = rocks_str_raw.split("\n\n")

In [4]:
rocks: list[Rock] = []
for rock_str in rocks_str:
    rocks.append(Rock())
    split_rock_str = rock_str.splitlines()
    for part_row, rock_line in enumerate(split_rock_str):
        for part_col, part, in enumerate(list(rock_line)):
            if part == ".":
                continue
            rocks[-1].add_part((part_col, len(split_rock_str) - part_row - 1))

# Input Processing

In [5]:
jet_str = open("data/17.txt", "r").read().strip()

In [6]:
jets: list[int] = []  # -1 (left) or 1 (right)
for char in list(jet_str):
    jets.append(-1 if char == "<" else 1)

# Part 1

In [7]:
rock_locations: set[tuple[int, int]] = set()
chamber_width = 7
max_rocks = 2022
tallest_rock = 0  # floor
current_rock_index = 0
current_jet_index = 0
current_rock_location: tuple[int, int] = (2, tallest_rock + 3)
num_rocks = 0

while num_rocks < max_rocks:
    # Process jet
    valid = True
    ## Create test rock
    future_rock_location = (
        current_rock_location[0] + jets[current_jet_index],
        current_rock_location[1]
    )
    current_jet_index = (current_jet_index + 1) % len(jets)
    ## Test wall collision
    if future_rock_location[0] < 0 or future_rock_location[0] + rocks[current_rock_index].width - 1 >= chamber_width:
        valid = False
    ## Test rock collisions
    for test_part in rocks[current_rock_index].parts:
        if (
                test_part[0] + future_rock_location[0],
                test_part[1] + future_rock_location[1]
        ) in rock_locations:
            valid = False
            break
    ## Set position if jet is valid
    if valid:
        current_rock_location = future_rock_location

    # Process gravity
    valid = True
    ## Create test rock
    future_rock_location = (
        current_rock_location[0],
        current_rock_location[1] - 1
    )
    ## Test floor collision
    if future_rock_location[1] < 0:
        valid = False
    ## Test rock collisions
    for test_part in rocks[current_rock_index].parts:
        if (
                test_part[0] + future_rock_location[0],
                test_part[1] + future_rock_location[1]
        ) in rock_locations:
            valid = False
            break
    ## Set position if jet is valid
    if valid:
        current_rock_location = future_rock_location
        continue

    # If gravity isn't valid, rock has settled, switch rock
    ## Apply rock part locations to settled rock set
    for new_part in rocks[current_rock_index].parts:
        rock_locations.add((
            new_part[0] + current_rock_location[0],
            new_part[1] + current_rock_location[1]
        ))
    ## Apply the tallest rock if possible
    tallest_rock = max(current_rock_location[1] + rocks[current_rock_index].height, tallest_rock)
    ## Increment current rock index
    current_rock_index = (current_rock_index + 1) % len(rocks)
    ## Update current rock location to new spawned rock
    current_rock_location = (2, tallest_rock + 3)
    ## Increment the number of rocks settled
    num_rocks += 1

In [8]:
tallest_rock

3100

# Part 2

In [9]:
import re
cycle_detector = re.compile(r"(((?:\(\d+,\d+,\d+\))+?)(\2)+)") # search
height_extractor = re.compile(r"\((\d+),\d+,\d+\)") # find_all of substring

In [10]:
rock_locations: set[tuple[int, int]] = set()
chamber_width = 7
max_rocks = 1000000000000
tallest_rock = 0  # floor
current_rock_index = 0
current_jet_index = 0
current_rock_location: tuple[int, int] = (2, tallest_rock + 3)
num_rocks = 0
history = ""

cycle_check_interval = 10_000
cycle_match = None
last_tallest: list[int] = []

while num_rocks < max_rocks:
    # Process jet
    valid = True
    ## Create test rock
    future_rock_location = (
        current_rock_location[0] + jets[current_jet_index],
        current_rock_location[1]
    )
    current_jet_index = (current_jet_index + 1) % len(jets)
    ## Test wall collision
    if future_rock_location[0] < 0 or future_rock_location[0] + rocks[current_rock_index].width - 1 >= chamber_width:
        valid = False
    ## Test rock collisions
    for test_part in rocks[current_rock_index].parts:
        if (
                test_part[0] + future_rock_location[0],
                test_part[1] + future_rock_location[1]
        ) in rock_locations:
            valid = False
            break
    ## Set position if jet is valid
    if valid:
        current_rock_location = future_rock_location

    # Process gravity
    valid = True
    ## Create test rock
    future_rock_location = (
        current_rock_location[0],
        current_rock_location[1] - 1
    )
    ## Test floor collision
    if future_rock_location[1] < 0:
        valid = False
    ## Test rock collisions
    for test_part in rocks[current_rock_index].parts:
        if (
                test_part[0] + future_rock_location[0],
                test_part[1] + future_rock_location[1]
        ) in rock_locations:
            valid = False
            break
    ## Set position if jet is valid
    if valid:
        current_rock_location = future_rock_location
        continue

    # If gravity isn't valid, rock has settled, switch rock
    ## Apply rock part locations to settled rock set
    for new_part in rocks[current_rock_index].parts:
        rock_locations.add((
            new_part[0] + current_rock_location[0],
            new_part[1] + current_rock_location[1]
        ))
    ## Apply the tallest rock if possible
    last_tallest.append(tallest_rock)
    tallest_rock = max(current_rock_location[1] + rocks[current_rock_index].height, tallest_rock)
    ## Update the history
    history += f"({tallest_rock - last_tallest[-1]},{current_rock_index},{current_jet_index})"
    ## Try to detect a cycle
    if num_rocks % cycle_check_interval == 0:
        cycle_match = cycle_detector.search(history)
        if cycle_match:
            break
    ## Increment current rock index
    current_rock_index = (current_rock_index + 1) % len(rocks)
    ## Update current rock location to new spawned rock
    current_rock_location = (2, tallest_rock + 3)
    ## Increment the number of rocks settled
    num_rocks += 1

# Check if we've exited because of a cycle detection
if cycle_match:
    # Begin extrapolating
    ## Compute start and end of data
    start_str = cycle_match.start()
    precycle_height_changes = height_extractor.findall(history[:start_str])
    cycle_height_changes_str = height_extractor.findall(cycle_match.group(2))
    end_str = start_str + len(cycle_match.group(2))

    cycle_height_changes = [int(x) for x in cycle_height_changes_str]
    cycle_sum = sum(cycle_height_changes)

    start = len(precycle_height_changes)
    cycle_len = len(cycle_height_changes)
    remaining = max_rocks - start
    full_cycles = remaining // cycle_len

    tallest_rock = last_tallest[start]
    tallest_rock += full_cycles * cycle_sum
    remaining = max_rocks - start - full_cycles * cycle_len
    tallest_rock += sum(cycle_height_changes[:remaining])

In [11]:
tallest_rock

1540634005751